In [16]:
import pandas as pd
import sqlite3
import os
import matplotlib.pyplot as plt
import numpy as np
from dbl_constants import airlines, lang_to_tag, tag_to_lang
import re
from datetime import datetime

database_path = "temporary_database.sqlite3"
conn = sqlite3.connect(database_path)
cur = conn.cursor()

cur.execute("""PRAGMA table_info(TWEETS);""")
tweet_columns = tuple(t[1] for t in cur.fetchall())
cur.execute("""PRAGMA table_info(USERS);""")
user_columns = tuple(t[1] for t in cur.fetchall())
cur.execute("""PRAGMA table_info(INTERACTIONS);""")
interaction_columns = tuple(t[1] for t in cur.fetchall())

In [19]:
# constants and auxilliary functions
date_range = []
cur.execute("""SELECT created_at FROM TWEETS order by created_at asc  limit 1""")
date_range.append(cur.fetchall())
cur.execute("""SELECT created_at FROM TWEETS order by created_at desc limit 1""")
date_range.append(cur.fetchall())
date_range = tuple(datetime.strptime(i[0][0], '%Y-%m-%d %H:%M:%S') for i in date_range)

def str_to_readable(s:str):
    return re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', s.replace('_',' '))
print(str_to_readable("Test_TestTest"))
print(str_to_readable("KLM"))


ValueError: time data 'Fri Aug 02 00:00:04 +0000 2019' does not match format '%Y-%m-%d %H:%M:%S'



# 1. Sprint 1 plots testing

At least 4 good plots. E.g. #tweets per
language, tweets sent/received per airline,
tweets per hour of day/week, per day of
week/year, per timezone, etc...


## 1.1 getting dataframes

In [6]:
# Amount per language per airline
dfs = []
for air_name in airlines:
    print(air_name, end=" ")
    airline_id = airlines[air_name]
    cur.execute("""
    SELECT
      lang,
      COUNT(*) AS `num`
    FROM
      TWEETS
    WHERE user_id == ?
    GROUP BY
      lang
    ORDER BY
      num DESC""", [airline_id])
    df_lang = pd.DataFrame(cur.fetchall(), columns=("lang", "count"))
    df_lang["airline"] = air_name
    dfs.append(df_lang)
    print("done")
df_languages_by_airline = pd.concat(dfs)
#df_languages_by_airline.set_index([df_languages_by_airline.index, "airline"], inplace=True)
df_languages_by_airline.sort_values("count", ascending=False, inplace=True)
print(df_languages_by_airline)

KLM done
AirFrance done
British_Airways done
AmericanAir done
Lufthansa done
AirBerlin done
AirBerlin assist done
easyJet done
RyanAir done
SingaporeAir done
Qantas done
EtihadAirways done
VirginAtlantic done
   lang   count          airline
0    en  114294      AmericanAir
0    en  106493  British_Airways
0    en   48423          easyJet
0    en   22941              KLM
0    en   21245   VirginAtlantic
..  ...     ...              ...
17   ja       1          easyJet
18   hu       1          easyJet
19   cy       1          easyJet
20   cs       1          easyJet
11   is       1   VirginAtlantic

[138 rows x 3 columns]


## 1.2 using plotly

In [ ]:
### Testing plotly
### plotly is not included in default anaconda environment so install it yourself
# plotly may be installed using 
# pip: $ pip install plotly==5.7.0
# or
# conda: $ conda install -c plotly plotly=5.7.0 
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.core.display import display, HTML
    
# prevent scrollbar

In [15]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 120;

<IPython.core.display.Javascript object>

In [11]:
# plot of tweets per airline per language
df_airlang = df_languages_by_airline.astype({"count":int}) 
df_airlang["log_count"] = np.log2(df_airlang["count"])
df_airlang["airline"] = df_airlang["airline"].apply(lambda x: str_to_readable(x))
df_airlang["lang"] = df_airlang["lang"].replace(tag_to_lang)

NameError: name 'str_to_readable' is not defined

In [12]:
# main pie
fig1 = px.pie(df_airlang, values="count",names="airline", title="Airlines")
fig1.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20, )
fig1.show()

NameError: name 'px' is not defined

In [13]:
df_total_counts = df_airlang.groupby("airline").sum().reset_index()
fig = px.scatter(df_total_counts, x="airline",y="airline", size="count", color="count", size_max=60,
                text='airline', title="Total amount of tweets per airlines")
fig.show()

NameError: name 'px' is not defined

In [9]:
# fig = px.scatter(
#     data_frame= df, 
#     x="airline", 
#     y="lang", 
#     size="count", 
#     color="count",
#     hover_name="airline",
#     #facet_col="lang",
#     #facet_col_wrap=2,
#     size_max=30,
    
#     title = "Interactive Plot of Tweet counts per language and airline",
#     labels = {"lang":"Language", "airline": "Airline", "count":"Tweet Count"}
# )
# pie plots
distr = (5, 3)

specs = [[{'type':'domain'} for i in range(distr[0])] for i in range(distr[1])]

fig = make_subplots(rows=distr[1], cols=distr[0], specs=specs)
col = 1
row = 1
for airline in df_airlang["airline"].unique():
    df_air = df_airlang[df_airlang["airline"]==airline].query("count > 250")
    if col > distr[0]:
        col = 1
        row += 1
    fig.add_trace(
        go.Pie(values=df_air["count"], labels=df_air["lang"], title=airline, automargin=True, hole=0.75, name=airline
        ), row, col)
    col += 1
fig.update_layout(
    height= 600,
    width = 1000
    #paper_bgcolor="LightSteelBlue",
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.3,
    xanchor="left",
    x=0.3,
    title="Tweets per language per airline"
))
fig.show()

In [10]:
### sunburst plot
unique_airlines = list(df_airlang["airline"].unique())
df_airlang["all"] = "<b>Total Airline Tweets <br> "+str(df_airlang["count"].sum())
df_airlang["langcount"] = df_airlang.agg('{0[lang]}: {0[count]}'.format, axis=1)

fig2 = px.sunburst(df_airlang, path=['all', 'airline', 'langcount'], values='count',
                    color='count', 
                    hover_name=df_airlang["airline"],
                    hover_data={"langcount": False},
                    color_continuous_scale='RdBu',
                    color_continuous_midpoint=np.average(df_airlang['count'], weights=df_airlang['count']),
                    width=900,
                    height=900,
                    title="Amount of tweets per airline and language <br>        From %s up to %s" 
                           % tuple(datetime.strftime(date, '%Y-%m-%d') for date in date_range))
fig2.update_traces(hovertemplate="""<b>%{label}</b>""")
fig2.show()


In [11]:
# singular plots
for airline in unique_airlines:
    df_seperate = df_airlang[df_airlang["airline"] == airline].copy()
    df_seperate["all"] = str(airline) + "<br>"+str(df_seperate["count"].sum()) + " Tweets"
    figd = px.sunburst(df_seperate, path=["all", 'lang', 'count'], values='count',
                        color='count',
                        color_continuous_scale='RdBu',
                        color_continuous_midpoint=np.average(df_airlang['count'], weights=df_airlang['count']),
                        width=900,
                        height=900,)
    figd.show()

In [12]:
### get all KLM tweets
cur.execute("""
    SELECT
      created_at, id
    FROM
      TWEETS
    WHERE user_id == %s"""% airlines["KLM"])
df_klm = pd.DataFrame(cur.fetchall(), columns = ["created_at", "id"])
df_klm['created_at'] = pd.to_datetime(df_klm['created_at'], format='%Y-%m-%d %H:%M:%S')
print(df_klm)

               created_at                   id
0     2019-05-22 12:23:03  1131173625258815488
1     2019-05-22 13:09:16  1131185259289206785
2     2019-05-22 13:20:05  1131187978771025921
3     2019-05-22 13:20:17  1131188028528107521
4     2019-05-22 13:20:31  1131188089328734209
...                   ...                  ...
34246 2020-03-30 18:11:29  1244688715806539776
34247 2020-03-30 18:15:32  1244689736125624321
34248 2020-03-30 18:17:19  1244690181963354118
34249 2020-03-30 18:18:55  1244690584759078916
34250 2020-03-30 18:33:15  1244694194708533253

[34251 rows x 2 columns]


In [13]:
airlines_dfs = []
for air_name in airlines:
    print(air_name, end=" ")
    airline_id = airlines[air_name]
    cur.execute("""
    SELECT
      *
    FROM
      TWEETS
    WHERE user_id == ?""", [airline_id])
    subdf = pd.DataFrame(cur.fetchall(), columns=tweet_columns)
    subdf["airline"] = air_name
    airlines_dfs.append(subdf)
    print("done")
df_airline_tweets = pd.concat(airlines_dfs)
df_airline_tweets['created_at'] = pd.to_datetime(df_airline_tweets['created_at'], format='%Y-%m-%d %H:%M:%S')
df_airline_tweets['day'] = df_airline_tweets.created_at.dt.dayofyear + \
        365 * (df_airline_tweets.created_at.dt.year - date_range[0].year)


KLM done
AirFrance done
British_Airways done
AmericanAir done
Lufthansa done
AirBerlin done
AirBerlin assist done
easyJet done
RyanAir done
SingaporeAir done
Qantas done
EtihadAirways done
VirginAtlantic done


In [14]:
df = df_airline_tweets.copy()

df_per_day_per_airline = df.groupby(["day", "airline"]).agg('count').unstack(level=1)["created_at"]

series = []
for airline in df_per_day_per_airline.columns:
    df = pd.DataFrame(df_per_day_per_airline[airline]).rename(columns={airline:"count"})
    df["airline"] = airline
    series.append(df)
plot_df = pd.concat(series).reset_index()
print(plot_df)
    
    

      day  count    airline
0     142   17.0  AirFrance
1     143   30.0  AirFrance
2     144   28.0  AirFrance
3     145   30.0  AirFrance
4     146   23.0  AirFrance
...   ...    ...        ...
3152  451  136.0    easyJet
3153  452  135.0    easyJet
3154  453   94.0    easyJet
3155  454   44.0    easyJet
3156  455  492.0    easyJet

[3157 rows x 3 columns]


In [15]:
fig = go.Figure()
fig = px.line(plot_df, x="day", y="count", color="airline", title="Tweets per day per airline")
fig.show()

In [3]:
cur.close()
conn.close()